<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Semantic_Features_3DVision_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our mapping pipeline is of the following structure:

1.   Detection of objects of certain object classes (e.g. traffic sign). Output: object bounding boxes
2.   Triangulation of objects. Output: object position relative to pose
3.   Creating map of objects. (And refining with filter, BA, etc.) Output: List of objects and their corresponding positions
4.   Visualizing map.





We start by loading the benchmark dataset. The Cityscapes dataset is used. 

Scripts for analyzing the dataset can be found here: https://github.com/mcordts/cityscapesScripts

How to download the zip files directly: https://towardsdatascience.com/download-city-scapes-dataset-with-script-3061f87b20d7



Download CityScape files: 

*   leftImg8bit_trainextra.zip
*   disparity_trainextra.zip
*   camera_trainextra.zip
*   vehicle_trainextra.zip

The files are unzipped into data/... respectively.









In [1]:
# Download and unzip (run only once!).
trajectoryname = "schweinfurt"

# Remove data directory if it is already loaded (if needed).
#!rm -r data
# Login.
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=ftaubner@ethz.ch&password=semantic_dudes&submit=Login' https://www.cityscapes-dataset.com/login/
# Get left camera images. I put it on OneDrive so that only Schweinfurt has to be downloaded.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4" -O leftImg8bit_trainextra.zip
# Alternatively, it can be downloaded from CityScapes website directly:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=4)
# Extract.
!mkdir data
!unzip leftImg8bit_trainextra.zip 'leftImg8bit/train_extra/schweinfurt/*' -d data
# And delete.
!rm leftImg8bit_trainextra.zip

# Get disparity maps.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199735&authkey=AEwGRlH_TySnyPM" -O disparity_trainextra.zip
# Original file:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=22)
!unzip disparity_trainextra.zip 'disparity/train_extra/schweinfurt/*' -d data
!rm disparity_trainextra.zip
# Get camera intrinsics. 
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=9
!unzip camera_trainextra.zip 'camera/train_extra/schweinfurt/*' -d data
!rm camera_trainextra.zip
# Get vehicle odometry.
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=11
!unzip vehicle_trainextra.zip 'vehicle/train_extra/schweinfurt/*' -d data
!rm vehicle_trainextra.zip

--2020-03-21 19:02:36--  https://www.cityscapes-dataset.com/login/
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cityscapes-dataset.com/downloads/ [following]
--2020-03-21 19:02:37--  https://www.cityscapes-dataset.com/downloads/
Reusing existing connection to www.cityscapes-dataset.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ]  42.77K  --.-KB/s    in 0.1s    

2020-03-21 19:02:39 (389 KB/s) - ‘index.html’ saved [43792]

--2020-03-21 19:02:40--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)

We start with the detection of objects of interest. Faster R-CNN is used for this task. (Julius)

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.4+cu100 in /usr/local/lib/python3.6/dist-packages (1.4.0+cu100)
Requirement already up-to-date: torchvision==0.5+cu100 in /usr/local/lib/python3.6/dist-packages (0.5.0+cu100)
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8h9u6ipb
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8h9u6ipb
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275366 sha256=b681b1229cf6c944e13d395a4697f5f1162176cd761365c0d41d430cf7b7bb6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-zv9ly4di/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Loading an on COCO pre-trained Faster R-CNN 


Installing Seamless Scene Segmentation from https://github.com/mapillary/seamseg
For Seamless Scene Segmentation a pre-trained version on the Mapillary Dataset exists. Dataloaders for Cityscapes seem to exist, too.

It is a network for panoptic segmentation basen on Mask-R-CNN. Alternatively the panoptic variant in the detectron2 reopisitory can also be trained on Cityscapes or Mapillary. Dataloaders seem to exist for both Datasets. 

In [0]:
#@title
!pip install git+https://github.com/mapillary/seamseg.git
!pip install wget

url = 'https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view'
wget.download(url)


These functions convert a difference in coordinates (lat., long.) to a difference in metric frame (x, y) and vice versa. Both conversions are dependent on the current latitude.
Under the assumption that we are in Europe, east of 0°!

Approximative conversions:

* Latitude: 1 deg = 110.574 km
* Longitude: 1 deg = 111.320*cos(latitude) km


In [0]:
import numpy as np

# Calculates the difference between two coordinates in meters (x to east, y to north).
def coord_diff_to_metric_diff(d_lat, d_long, lat):
  d_y = d_lat * 110574
  d_x = d_long * 111320 * np.cos(np.radians(lat))
  return d_x, d_y

# Converts the metric difference to differences in latitude and longitude.
def metric_diff_to_coord_diff(d_x, d_y, lat):
  d_lat = d_y / 110574
  d_long = d_x / 111320 / np.cos(np.radians(lat))
  return d_lat, d_long

In [0]:
# Testing and example (taken from Johannes' map pic on WhatsApp):
# Converting coords into metrics:
d_x, d_y = coord_diff_to_metric_diff(48.370-48.369, 10.896-10.894, 48.368)
print("Delta x in meters: {}".format(d_x))
print("Delta y in meters: {}".format(d_y))

# Converting back to coords: should be the same:
d_lat, d_long = metric_diff_to_coord_diff(d_x, d_y, 48.368)
print("Difference in latitude: {}".format(d_lat))
print("Difference in longitude: {}".format(d_long))